In [2]:
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

pindir="../pin_experiments"
#querydir="query1_cores16_frate100000_6000000_fbuff-1_itr1_ondemanddvfs1_source16_mapper16_sink16_repeat0"
configs = ["base", "C1", "C2", "C3", "C4", "C5"]

df_dict = {
    "config" : [],
    "pkgPower" : [],
    "pkgPowerstd" : [],
    "dramPower" : [],
    "dramPowerstd" : [],
    "SourceOut" : [],
    "POLL" : [],
    "C1" : [],
    "C1E" : [],
    "C3" : [],
    "C6" : [],
    "cpu_user" : [],
    "cpu0_user": [],
    "cpu1_user": [],
    "cpu2_user": [],
    "cpu3_user": [],
    "cpu4_user": [],
    "cpu5_user": [],
    "cpu6_user": [],
    "cpu7_user": [],
    "cpu8_user": [],
    "cpu9_user": [],
    "cpu10_user": [],
    "cpu11_user": [],
    "cpu12_user": [],
    "cpu13_user": [],
    "cpu14_user": [],
    "cpu15_user": [],
    "cpu_idle" : [],
    "cpu0_idle": [],
    "cpu1_idle": [],
    "cpu2_idle": [],
    "cpu3_idle": [],
    "cpu4_idle": [],
    "cpu5_idle": [],
    "cpu6_idle": [],
    "cpu7_idle": [],
    "cpu8_idle": [],
    "cpu9_idle": [],
    "cpu10_idle": [],
    "cpu11_idle": [],
    "cpu12_idle": [],
    "cpu13_idle": [],
    "cpu14_idle": [],
    "cpu15_idle": []
}

def resetdict():
    global df_dict
    
    df_dict = {
    "config" : [],
    "pkgPower" : [],
    "pkgPowerstd" : [],
    "dramPower" : [],
    "dramPowerstd" : [],
    "SourceOut" : [],
    "POLL" : [],
    "C1" : [],
    "C1E" : [],
    "C3" : [],
    "C6" : [],
    "cpu_user" : [],
    "cpu0_user": [],
    "cpu1_user": [],
    "cpu2_user": [],
    "cpu3_user": [],
    "cpu4_user": [],
    "cpu5_user": [],
    "cpu6_user": [],
    "cpu7_user": [],
    "cpu8_user": [],
    "cpu9_user": [],
    "cpu10_user": [],
    "cpu11_user": [],
    "cpu12_user": [],
    "cpu13_user": [],
    "cpu14_user": [],
    "cpu15_user": [],
    "cpu_idle" : [],
    "cpu0_idle": [],
    "cpu1_idle": [],
    "cpu2_idle": [],
    "cpu3_idle": [],
    "cpu4_idle": [],
    "cpu5_idle": [],
    "cpu6_idle": [],
    "cpu7_idle": [],
    "cpu8_idle": [],
    "cpu9_idle": [],
    "cpu10_idle": [],
    "cpu11_idle": [],
    "cpu12_idle": [],
    "cpu13_idle": [],
    "cpu14_idle": [],
    "cpu15_idle": []
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def run(querydir):
    resetdict()
    
    for c in configs:
        fname = f"{pindir}/{querydir}/{c}"
    
        df_dict['config'].append(c)
        
        # Power
        pkgs=[]
        dram=[]
        with open(f"{fname}/rapl.log") as f:
            lines = f.readlines()
        for line in lines:
            pkgs.append(float(line.strip().split(' ')[0]))
            dram.append(float(line.strip().split(' ')[1]))
        #print(f"CPU pkg: {np.mean(np.array(pkgs))} {np.std(np.array(pkgs))}")
        #print(f"DRAM pkg: {np.mean(np.array(dram))} {np.std(np.array(dram))}")
        df_dict['pkgPower'].append(np.mean(np.array(pkgs)))
        df_dict['pkgPowerstd'].append(np.std(np.array(pkgs)))
        df_dict['dramPower'].append(np.mean(np.array(dram)))
        df_dict['dramPowerstd'].append(np.std(np.array(dram)))
    
        # Source records out
        kwlist={'numRecordsOutPerSecond':[]}
    
        with open(f"{fname}/Source.0") as f:
            lines = f.read()
        for lc in lines.split('; '):
            for kw in kwlist.keys():
                if(kw in lc):
                    ldict=eval(lc.replace('[','').replace(']',''))
                    sourceout = float(ldict['value'])
        #print(f"Source out: {sourceout}")
        df_dict['SourceOut'].append(sourceout)
        
        # idle stats
        statsSTART = pd.read_csv(f"{fname}/statsSTART.csv", names=['POLL', 'C1', 'C1E', 'C3', 'C6', 
                                                           'RXP', 'RXB', 'TXP', 'TXB', 'ERXP', 'ERXB', 'ETXP', 'ETXB'])
        statsEND = pd.read_csv(f"{fname}/statsEND.csv", names=['POLL', 'C1', 'C1E', 'C3', 'C6', 
                                                           'RXP', 'RXB', 'TXP', 'TXB', 'ERXP', 'ERXB', 'ETXP', 'ETXB'])
        statsdf = pd.concat([statsSTART, statsEND])
        statsdf = statsdf.diff()
        statsdf.dropna(inplace=True)
        df_dict['POLL'].append(statsdf['POLL'].values[0])
        df_dict['C1'].append(statsdf['C1'].values[0])
        df_dict['C1E'].append(statsdf['C1E'].values[0])
        df_dict['C3'].append(statsdf['C3'].values[0])
        df_dict['C6'].append(statsdf['C6'].values[0])
        #print(statsdf[['POLL', 'C1', 'C3', 'C1E', 'C6']])
    
        # /proc/stat
        with open(f"{fname}/procstatsSTART.csv") as f:
            lines = f.readlines()
    
        procstats={}
        for line in lines:
            if 'cpu' in line:
                stats = line.strip().split(' ')
                #print(stats)
                tmp = []
                for i in range(1, len(stats)):
                    if len(stats[i]) > 0:
                        tmp.append(int(stats[i]))
                procstats[stats[0]] = tmp
                #print(stats[0], len(tmp))
        
        with open(f"{fname}/procstatsEND.csv") as f:
            lines = f.readlines()
        for line in lines:
            if 'cpu' in line:
                stats = line.strip().split(' ')
                tmp = []
                #print(stats)
                for i in range(1, len(stats)):
                    if len(stats[i]) > 0:
                        tmp.append(int(stats[i]))
                for i in range(0, len(tmp)):
                    procstats[stats[0]][i] = tmp[i] - procstats[stats[0]][i]
        #print(procstats)  
    
        for k, v in procstats.items():
            df_dict[f"{k}_user"].append(v[0])
            df_dict[f"{k}_idle"].append(v[3])
    
    dd1 = pd.DataFrame(df_dict)
    #print(dd1.columns)
    return dd1


In [16]:
dd = run("query1_cores16_frate100000_6000000_fbuff-1_itr1_ondemanddvfs1_source16_mapper16_sink16_repeat0")
dd[['config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
0,base,51.044972,10.841678,100000.0,435.0,688222.0,52530.0,0.0,248992.0,41686,146579
1,C1,49.709064,10.682763,100000.0,172.0,152983.0,33600.0,0.0,143947.0,41442,146842
2,C2,50.330844,10.740796,100000.0,119.0,148870.0,30253.0,0.0,142878.0,41447,146779
3,C3,49.561898,10.470423,100000.0,421.0,715578.0,46908.0,0.0,196903.0,41533,146968
4,C4,51.535734,10.562459,100000.0,671.0,1119410.0,29674.0,0.0,163613.0,41810,145921
5,C5,52.261679,10.660946,100000.0,683.0,1134731.0,29680.0,0.0,167907.0,41586,146362


In [17]:
dd = run("query1_cores16_frate200000_6000000_fbuff-1_itr1_ondemanddvfs1_source16_mapper16_sink16_repeat0")
dd[['config', 'pkgPower', 'dramPower',  'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
0,base,85.950764,13.018429,200000.000000,772.0,1026312.0,115153.0,0.0,338722.0,97424,87410
1,C1,78.453991,12.647399,177365.933333,153.0,14233.0,2696.0,0.0,50043.0,88558,96412
2,C2,85.365240,13.383369,189034.450000,779.0,1397695.0,36169.0,0.0,2015285.0,89790,92196
3,C3,93.307821,13.854962,200000.000000,1710.0,2819661.0,235617.0,0.0,635727.0,96944,85901
4,C4,88.501540,13.685402,195541.000000,903.0,1153644.0,357821.0,0.0,461478.0,95114,86799
5,C5,92.177424,13.192093,199704.983333,1721.0,2665088.0,72605.0,0.0,184932.0,96389,87300


In [18]:
dd = run("query1_cores16_frate300000_6000000_fbuff-1_itr1_ondemanddvfs1_source16_mapper16_sink16_repeat0")
dd[['config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
0,base,97.891539,14.728425,300014.000000,1053.0,1631042.0,95913.0,0.0,247219.0,123258,58846
1,C1,76.211965,13.264201,212614.850000,135.0,23749.0,4209.0,0.0,56484.0,87514,96729
2,C2,78.268444,13.230892,217333.616667,62.0,21217.0,3829.0,0.0,66691.0,87740,96712
3,C3,99.335712,14.954621,299964.800000,1267.0,2527175.0,91851.0,0.0,238997.0,122473,58816
4,C4,89.235918,13.716309,246395.700000,1331.0,2834309.0,28100.0,0.0,137220.0,98807,83675
5,C5,91.091564,13.501470,245300.133333,1541.0,2891612.0,29462.0,0.0,145117.0,98760,83797


In [19]:
dd = run("query1_cores16_frate400000_6000000_fbuff-1_itr1_ondemanddvfs1_source16_mapper16_sink16_repeat0")
dd[['config', 'pkgPower', 'dramPower', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

,config,pkgPower,dramPower,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
0,base,113.372761,15.629505,382755.083333,321.0,470386.0,47431.0,0.0,157192.0,167664,11659
1,C1,75.898671,12.736778,197592.416667,147.0,17390.0,3434.0,0.0,51793.0,88368,96487
2,C2,81.405890,13.388156,197546.333333,586.0,446653.0,36115.0,0.0,1414860.0,89308,93138
3,C3,112.139261,14.930788,360877.650000,586.0,565116.0,33530.0,0.0,163970.0,166511,14744
4,C4,86.538924,13.757331,211101.266667,1070.0,2511032.0,108537.0,0.0,200221.0,95972,85770
5,C5,89.852752,12.928493,214446.016667,1475.0,2707664.0,50192.0,0.0,168046.0,98709,84582
